In [1]:
import os
import pickle as pkl
import numpy as np
import pandas as pd
import joblib

# set new path
os.chdir('/home/cyprien/CrystaLLMv2_PKV')

from _utils import build_challenge_dataframe, filter_df_to_context, prompts_for_challenge_set

/home/cyprien/miniconda3/envs/crystallmv2_venv/lib/python3.10/site-packages/material_hasher/hasher/__init__.py:21: ImportWarning: Failed to import SLICES. If you would like to use this module, please consider running uv pip install -r requirements_slices.txt
  warnings.warn(


In [ ]:
# load challenge_set_v1.zip
with open('challenge_set_v1.zip', 'rb') as f:
    # unzip file with unzip
    import zipfile
    with zipfile.ZipFile(f, 'r') as zip_ref:
        zip_ref.extractall('challenge_set_v1')


In [ ]:
build_challenge_dataframe(
    '_utils/_evaluation_og/challenge_set/files',
    '_utils/_evaluation_og/challenge_set/train.parquet'
)

Saved 70 records to _utils/_evaluation_og/challenge_set/train.parquet


In [8]:
!python -m _utils._preprocessing._cleaning \
    _utils/_evaluation_og/challenge_set/train.parquet \
    --out _utils/_evaluation_og/challenge_set/clean.parquet

Loading data from _utils/_evaluation_og/challenge_set/train.parquet as Parquet with zstd compression...

==Lets augment the CIFs now (parrallelising sometimes takse a min to kick in :)==
Number of CIFs before preprocessing: 70
Number of workers: 4
100%|██████████████████████████████████████████| 70/70 [00:00<00:00, 801.34it/s]
Number of CIFs before filtering out bad ones:  70
Number of CIFs after filtering: 70

Saving updated dataframe to _utils/_evaluation_og/challenge_set/clean.parquet as Parquet with zstd compression...
Preprocessing completed successfully.


In [ ]:
df = pd.read_parquet('_utils/_evaluation_og/challenge_set/clean.parquet')

filtered_df = filter_df_to_context(
    df,
    context=1024,
    cif_column="CIF"
)

print(f"Number of rows before filtering: {len(df)}")
print(f"Number of rows after filtering: {len(filtered_df)}")

filtered_df.to_parquet('_utils/_evaluation_og/challenge_set/clean_filtered.parquet', index=False)

Tokenizer validation passed: token vocabulary is consistent.
Number of rows before filtering: 70
Number of rows after filtering: 64


In [ ]:
filtered_df = pd.read_parquet('_utils/_evaluation_og_files/challenge_set/clean_filtered.parquet')

prompts = prompts_for_challenge_set(filtered_df, cif_column="CIF", prompt_column="Prompt", with_sg=False)

prompts_sg = prompts_for_challenge_set(filtered_df, cif_column="CIF", prompt_column="Prompt", with_sg=True)

# # Save the updated DataFrame with the new 'Prompt' column
prompts_sg.to_parquet('_utils/_evaluation_og/challenge_set/challenge_prompt_sg.parquet', index=False)
prompts.to_parquet('_utils/_evaluation_og/challenge_set/challenge_prompt.parquet', index=False)

In [4]:
df = pd.read_parquet('_utils/_evaluation_og_files/challenge_set/challenge_prompt_sg.parquet')
df

,Material ID,CIF,Prompt
0,KScP2O7,data_[K4Sc4P8O28]\nloop_\n _atom_type_symbol\n...,<bos>\ndata_[K4Sc4P8O28]\nloop_\n _atom_type_s...
1,Ca2Te3O8,data_[Ca24Te36O96]\nloop_\n _atom_type_symbol\...,<bos>\ndata_[Ca24Te36O96]\nloop_\n _atom_type_...
2,Ni3Te2O2(PO4)2(OH)4,data_[Ni6Te4P4H8O28]\nloop_\n _atom_type_symbo...,<bos>\ndata_[Ni6Te4P4H8O28]\nloop_\n _atom_typ...
3,Na3Te2(FeO4)3,data_[Na24Fe24Te16O96]\nloop_\n _atom_type_sym...,<bos>\ndata_[Na24Fe24Te16O96]\nloop_\n _atom_t...
4,Na6Li4WO4(CO3)4,data_[Na6Li4W1C4O16]\nloop_\n _atom_type_symbo...,<bos>\ndata_[Na6Li4W1C4O16]\nloop_\n _atom_typ...
...,...,...,...
59,Li9Al4Sn5,data_[Li9Al4Sn5]\nloop_\n _atom_type_symbol\n ...,<bos>\ndata_[Li9Al4Sn5]\nloop_\n _atom_type_sy...
60,Eu2FeGe2OS6,data_[Eu4Fe2Ge4S12O2]\nloop_\n _atom_type_symb...,<bos>\ndata_[Eu4Fe2Ge4S12O2]\nloop_\n _atom_ty...
61,Nd3BSi2O10,data_[Nd24Si16B8O80]\nloop_\n _atom_type_symbo...,<bos>\ndata_[Nd24Si16B8O80]\nloop_\n _atom_typ...
62,Na2Hf(BO3)2,data_[Na6Hf3B6O18]\nloop_\n _atom_type_symbol\...,<bos>\ndata_[Na6Hf3B6O18]\nloop_\n _atom_type_...


In [ ]:
!python _utils/_evaluation_og/generate_CIFs.py --config _config_files/og_eval/lematerial-challenge_eval.jsonc

In [ ]:
!python _utils/_evaluation_og/postprocess.py \
    --input_file '_utils/_evaluation_og_files/challenge_set/gen_07T10K.parquet' \
    --output_file '_utils/_evaluation_og_files/challenge_set/gen_07T10K_processed.parquet' \
    --num_workers 8

In [ ]:
!python _utils/_metrics/challenge_set_metrics.py \
    --input_parquet '_utils/_evaluation_og_files/challenge_set/filtered_sg_processed.parquet' \
    --path_to_db '_utils/_evaluation_og_files/challenge_set/train.parquet' \
    --num_gens 20 \
    --num_workers 16